# Final Project
Fall 2018

Xavier Hung

Weiyi Chen

### Problem: Do red light cameras result in more speeding?

In [445]:
import os
import pandas as pd
import numpy as np
import math
from ast import literal_eval
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.annotations import Title
from bokeh.tile_providers import CARTODBPOSITRON
import warnings
warnings.filterwarnings('ignore')

redlight_loc = pd.read_csv('red-light-camera-locations.csv')
redlight_violations = pd.read_csv('red-light-camera-violations.csv')

speed_loc = pd.read_csv('speed-camera-locations.csv')
speed_violations = pd.read_csv('speed-camera-violations.csv')

In [446]:
# plot red light camera and speed locations
speed_loc["COORDINATES"] = '('+ speed_loc["LATITUDE"].astype(str) +','+ speed_loc["LONGITUDE"].astype(str) + ')'
redlight_loc["COORDINATES"] = '('+ redlight_loc["LATITUDE"].astype(str) +','+ redlight_loc["LONGITUDE"].astype(str) + ')'

def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

speed_loc['coords_x'] = speed_loc['COORDINATES'].apply(lambda x: merc(x)[0])
speed_loc['coords_y'] = speed_loc['COORDINATES'].apply(lambda x: merc(x)[1])

redlight_loc['coords_x'] = redlight_loc['COORDINATES'].apply(lambda x: merc(x)[0])
redlight_loc['coords_y'] = redlight_loc['COORDINATES'].apply(lambda x: merc(x)[1])

p = figure(x_range=(-9790000, -9735000), y_range=(5105000, 5165000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

p.circle(x = speed_loc['coords_x'], y = speed_loc['coords_y'], legend = "Speed Camera", fill_color="#FF0000")
p.circle(x = redlight_loc['coords_x'], y = redlight_loc['coords_y'], legend = "Red Light Camera")

t = Title()
t.text = 'Locations of All Speed Cameras and Red Light Cameras'
p.title = t

p.legend.location = "top_right"
p.legend.click_policy="hide"

output_notebook()
show(p)

Loading BokehJS ...

In [457]:
# find red light cameras near speed cameras
#redlight_loc_copy = pd.DataFrame(columns = list(redlight_loc.columns.values))
#speed_loc_copy = pd.DataFrame(columns = list(speed_loc.columns.values))

for i in range(0,speed_loc.shape[0]):
    for j in range(0,redlight_loc.shape[0]):
        if abs(speed_loc.iloc[i,4] - redlight_loc.iloc[j,5]) < 0.0001 and abs(speed_loc.iloc[i,5] - redlight_loc.iloc[j,6] < 0.0001):
            redlight_loc_copy = redlight_loc_copy.append(redlight_loc.iloc[j], ignore_index = True)
            speed_loc_copy = speed_loc_copy.append(speed_loc.iloc[i], ignore_index = True)

In [458]:
remove = ["(Speed", "Camera)","Ave","ST","Rd"]
for i in range(0,speed_loc_copy.shape[0]):
    speed_loc_copy.iloc[i,0] = " ".join([word for word in speed_loc_copy.iloc[i,0].split() if word not in remove])

# subset dataframe where speed cameras are near red light cameras
#speed_violations_copy = pd.DataFrame(columns = list(speed_violations.columns.values))
for i in range(0,speed_loc_copy.shape[0]):
    address = speed_violations[speed_violations['ADDRESS'].str.contains(speed_loc_copy.iloc[i,0].upper())]
    speed_violations_copy = speed_violations_copy.append(address, ignore_index = True)

In [456]:
# subset dataframe where red light cameras are near speed cameras
speed_violations_copy['ADDRESS'].value_counts()

4042 W NORTH AVE         33966
6226 W IRVING PARK RD    26928
1334 W GARFIELD BLVD     26877
3450 W 71ST              26690
5454 W IRVING PARK       26146
4124 W FOSTER AVE        26112
3034 W FOSTER            23273
4040 W CHICAGO AVE       14161
5446 W FULLERTON         14144
7826 S PULASKI           13175
5428 S PULASKI           12376
4040 W 55TH              12342
4042 W ROOSEVELT RD      12138
3810 W BELMONT AVE        8551
819 E 71ST ST              189
Name: ADDRESS, dtype: int64

In [434]:
redlight_loc_copy["COORDINATES"] = '('+ redlight_loc_copy["LATITUDE"].astype(str) +','+ redlight_loc_copy["LONGITUDE"].astype(str) + ')'
speed_loc_copy["COORDINATES"] = '('+ speed_loc_copy["LATITUDE"].astype(str) +','+ speed_loc_copy["LONGITUDE"].astype(str) + ')'

speed_loc_copy['coords_x'] = speed_loc_copy['COORDINATES'].apply(lambda x: merc(x)[0])
speed_loc_copy['coords_y'] = speed_loc_copy['COORDINATES'].apply(lambda x: merc(x)[1])

redlight_loc_copy['coords_x'] = redlight_loc_copy['COORDINATES'].apply(lambda x: merc(x)[0])
redlight_loc_copy['coords_y'] = redlight_loc_copy['COORDINATES'].apply(lambda x: merc(x)[1])

p = figure(x_range=(-9780000, -9745000), y_range=(5120000, 5160000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

p.circle(x = redlight_loc_copy['coords_x'], y = redlight_loc_copy['coords_y'], legend = "Red Light Camera", fill_color="#FF0000")
p.circle(x = speed_loc_copy['coords_x'], y = speed_loc_copy['coords_y'], legend = "Speed Camera",)

t = Title()
t.text = 'Speed Cameras in Close Proximity to Red Light Cameras'
p.title = t

p.legend.location = "top_right"
p.legend.click_policy="hide"

output_notebook()
show(p)

Loading BokehJS ...

In [435]:
#find top three intersections with speed cameras nearby that have the most red light violations on a single day
redlight_violations_copy = redlight_violations_copy.sort_values('VIOLATIONS', ascending = False)
redlight_violations_copy.drop_duplicates(subset = 'INTERSECTION', keep = "first")

,INTERSECTION,CAMERA ID,ADDRESS,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
16047,WENTWORTH AND GARFIELD,2261.0,5500 S WENTWORTH AVEN,2016-06-11T00:00:00,68,NaN,NaN,NaN,NaN,NaN
15434,BELMONT AND KEDZIE,1372.0,3200 N KEDZIE AVE,2016-05-21T00:00:00,40,1.154427e+06,1.921110e+06,41.939352,-87.707847,"{'needs_recoding': False, 'longitude': '-87.70..."
186,IRVING PARK AND LARAMIE,1533.0,5200 W IRVING PARK ROA,2014-08-30T00:00:00,26,NaN,NaN,NaN,NaN,NaN
16895,HOMAN/KIMBALL AND NORTH,1773.0,3400 W NORTH AVE,2016-07-16T00:00:00,25,1.153387e+06,1.910466e+06,41.910163,-87.711954,"{'needs_recoding': False, 'longitude': '-87.71..."
18349,PULASKI AND FOSTER,1104.0,4000 W FOSTER AVE,2016-09-15T00:00:00,25,1.148759e+06,1.934294e+06,41.975641,-87.728335,"{'needs_recoding': False, 'longitude': '-87.72..."
18064,ROOSEVELT AND PULASKI,2314.0,4000 W ROOSEVELT ROAD,2016-09-04T00:00:00,19,1.149885e+06,1.894449e+06,41.866281,-87.725234,"{'needs_recoding': False, 'longitude': '-87.72..."
27729,PULASKI AND CHICAGO,1983.0,4000 W CHICAGO AVENUE,2017-08-06T00:00:00,18,1.149562e+06,1.905087e+06,41.895477,-87.726144,"{'needs_recoding': False, 'longitude': '-87.72..."
24198,LARAMIE AND FULLERTON,1424.0,5200 W FULLERTON AVENUE,2017-05-22T00:00:00,18,1.141306e+06,1.915503e+06,41.924216,-87.756211,"{'needs_recoding': False, 'longitude': '-87.75..."
24294,WESTERN AND FOSTER,1074.0,2400 W FOSTER AVE,2017-05-26T00:00:00,18,1.159379e+06,1.934503e+06,41.976000,-87.689275,"{'needs_recoding': False, 'longitude': '-87.68..."
35581,KEDZIE AND 71ST,2352.0,7100 S KEDZIE AVENUE,2018-06-24T00:00:00,15,1.156226e+06,1.857330e+06,41.764295,-87.702957,"{'needs_recoding': False, 'longitude': '-87.70..."


In [19]:
################# IGNORE
# find top 3 red light cameras that have the most violations
redlight_freq = redlight_violations['INTERSECTION'].value_counts()
redlight_freq.head()

CALIFORNIA AND DIVERSEY    4593
KOSTNER AND NORTH          4073
BELMONT AND KEDZIE         3899
STONEY ISLAND AND 79TH     3367
ROOSEVELT AND HALSTED      3161
Name: INTERSECTION, dtype: int64

In [20]:
# find top 3 speed cameras that have the most violations
speed_freq = speed_violations['ADDRESS'].value_counts()
speed_freq.head()

6226 W IRVING PARK RD    1584
1315 W GARFIELD BLVD     1584
536 E MORGAN DR          1584
2080 W PERSHING          1584
5520 S WESTERN           1584
Name: ADDRESS, dtype: int64

In [21]:
# data sets containing subsets of top 3 intersections with the most red light violations
redlight1 = redlight_violations.loc[redlight_violations['INTERSECTION'] == redlight_freq.axes[0][0]].reset_index(drop=True)
redlight2 = redlight_violations.loc[redlight_violations['INTERSECTION'] == redlight_freq.axes[0][1]].reset_index(drop=True)
redlight3 = redlight_violations.loc[redlight_violations['INTERSECTION'] == redlight_freq.axes[0][2]].reset_index(drop=True)

In [22]:
speed1 = speed_violations.loc[speed_violations['ADDRESS'] == speed_freq.axes[0][0]].reset_index(drop=True)
speed2 = speed_violations.loc[speed_violations['ADDRESS'] == speed_freq.axes[0][1]].reset_index(drop=True)
speed3 = speed_violations.loc[speed_violations['ADDRESS'] == speed_freq.axes[0][2]].reset_index(drop=True)
speed4 = speed_violations.loc[speed_violations['ADDRESS'] == speed_freq.axes[0][3]].reset_index(drop=True)
speed5 = speed_violations.loc[speed_violations['ADDRESS'] == speed_freq.axes[0][4]].reset_index(drop=True)

In [23]:
# map red light violations and speed violations to see if any overlap


In [24]:
# find speed cameras close to red light cameras

In [25]:
'''from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5'''

In [100]:
def merc(lat, lon):
    x = 6378137.000 * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return x, y

In [121]:
speed = speed1.copy().append(speed2).append(speed3).append(speed4).append(speed5)
speed = speed.drop_duplicates(subset=['LATITUDE', 'LONGITUDE']).reset_index(drop=True)

redlight = redlight1.copy().append(redlight2).append(redlight3)
redlight = redlight.drop_duplicates(subset=['LATITUDE', 'LONGITUDE']).reset_index(drop=True)

In [122]:
#speed.iloc[0][4], speed.iloc[0][5] = merc(speed.iloc[0][6], speed.iloc[0][7])
x,y = merc(speed.iloc[0][6], speed.iloc[0][7])
x1,y1 = merc(speed.iloc[1][6], speed.iloc[1][7])
x2,y2 = merc(speed.iloc[2][6], speed.iloc[2][7])
x3,y3 = merc(speed.iloc[3][6], speed.iloc[3][7])
x4,y4 = merc(speed.iloc[4][6], speed.iloc[4][7])

x5,y5 = merc(redlight.iloc[0][7], redlight.iloc[0][8])
x6,y6 = merc(redlight.iloc[1][7], redlight.iloc[1][8])
x7,y7 = merc(redlight.iloc[2][7], redlight.iloc[2][8])
x8,y8 = merc(redlight.iloc[3][7], redlight.iloc[3][8])
x9,y9 = merc(redlight.iloc[4][7], redlight.iloc[4][8])
x10,y10 = merc(redlight.iloc[5][7], redlight.iloc[5][8])
x11,y11 = merc(redlight.iloc[6][7], redlight.iloc[6][8])

In [111]:
speed['X COORDINATE'], speed['Y COORDINATE'] = speed[['LATITUDE', 'LONGITUDE']].apply(merc, axis=1)
#speed['Y COORDINATE'] = speed['LATITUDE'].apply(lambda x: merc(x)[1])
#merc(speed1.iloc[0,6], speed1.iloc[0,7])
#speed.iloc[0][6]
speed

TypeError: ("merc() missing 1 required positional argument: 'lon'", 'occurred at index 0')

In [124]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
p = figure(x_range=(-9780000, -9745000), y_range=(5130000, 5160000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)
p.circle(x = x, y = y, fill_color="#FF0000")
p.circle(x = x1, y = y1, fill_color="#FF0000")
p.circle(x = x2, y = y2, fill_color="#FF0000")
p.circle(x = x3, y = y3, fill_color="#FF0000")


p.circle(x = x5, y = y5)
p.circle(x = x6, y = y6)
p.circle(x = x7, y = y7)
p.circle(x = x8, y = y8)
p.circle(x = x9, y = y9)
p.circle(x = x10, y = y10)
p.circle(x = x11, y = y11)
output_notebook()
show(p)

Loading BokehJS ...

In [150]:
# plot red light camera and speed locations
speed_loc["COORDINATES"] = '('+ speed_loc["LATITUDE"].astype(str) +','+ speed_loc["LONGITUDE"].astype(str) + ')'
redlight_loc["COORDINATES"] = '('+ redlight_loc["LATITUDE"].astype(str) +','+ redlight_loc["LONGITUDE"].astype(str) + ')'

def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

speed_loc['coords_x'] = speed_loc['COORDINATES'].apply(lambda x: merc(x)[0])
speed_loc['coords_y'] = speed_loc['COORDINATES'].apply(lambda x: merc(x)[1])

redlight_loc['coords_x'] = redlight_loc['COORDINATES'].apply(lambda x: merc(x)[0])
redlight_loc['coords_y'] = redlight_loc['COORDINATES'].apply(lambda x: merc(x)[1])

from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
p = figure(x_range=(-9780000, -9745000), y_range=(5130000, 5160000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

p.circle(x = speed_loc['coords_x'], y = speed_loc['coords_y'], fill_color="#FF0000")
p.circle(x = redlight_loc['coords_x'], y = redlight_loc['coords_y'])

output_notebook()
show(p)

Loading BokehJS ...

In [201]:
for i in range(0,speed_loc.shape[0]-1):
    for j in range(0,redlight_loc.shape[0]-1):
        if abs(speed_loc.iloc[i,4] - redlight_loc.iloc[j,5]) < 0.0001 and abs(speed_loc.iloc[i,5] - redlight_loc.iloc[j,6] < 0.0001):
            print(speed_loc.iloc[i,0],redlight_loc.iloc[j,0])

7826 S Pulaski Rd (Speed Camera) Halsted-79th
3450 W 71ST ST (Speed Camera) Kedzie-71st
3034 W Foster Ave (Speed Camera) Western-Foster
4040 W 55th (Speed Camera) Pulaski-55th
5428 S Pulaski (Speed Camera) Wentworth-Garfield
5454 W Irving Park (Speed Camera) Laramie-Irving Park
1334 W Garfield Blvd (Speed Camera) Wentworth-Garfield
3810 W Belmont Ave (Speed Camera) Kedzie-Belmont
5446 W Fullerton (Speed Camera) Laramie-Fullerton
4042 W Roosevelt Rd (Speed Camera) Pulaski-Roosevelt
4124 W Foster Ave (Speed Camera) Pulaski-Foster
4040 W Chicago Ave (Speed Camera) Pulaski-Chicago
6226 W Irving Park Rd (Speed Camera) Austin-Irving Park
4042 W North Ave (Speed Camera) Homan-Kimball-North
4042 W North Ave (Speed Camera) Pulaski-North
819 E 71st St (Speed Camera) Cottage Grove-71st-South Chicago


In [202]:
redlight_violations_copy = redlight_violations.loc[redlight_violations['INTERSECTION'].isin(['79TH AND HALSTED', 'KEDZIE AND 71ST', 'WESTERN AND FOSTER', '55TH AND PULASKI', 'WENTWORTH AND GARFIELD', 'IRVING PARK AND LARAMIE', 'BELMONT AND KEDZIE', 'LARAMIE AND FULLERTON', 'ROOSEVELT AND PULASKI', 'PULASKI AND FOSTER', 'PULASKI AND CHICAGO', 'AUSTIN AND IRVING PARK', 'HOMAN/KIMBALL AND NORTH', 'PULASKI AND NORTH', 'COTTAGE GROVE AND 71ST'])].reset_index(drop=True)
speed_violations_copy = speed_violations.loc[speed_violations['ADDRESS'].isin(['7826 S PULASKI', '3450 W 71ST', '3034 W FOSTER', '4040 W 55TH', '5428 S PULASKI', '5454 W IRVING PARK', '1334 W GARFIELD BLVD', '3810 W BELMONT AVE', '5446 W FULLERTON', '4042 W ROOSEVELT RD', '4124 W FOSTER AVE', '4040 W CHICAGO AVE', '6226 W IRVING PARK RD', '4042 W NORTH AVE', '819 E 71ST ST'])].reset_index(drop=True)

In [207]:
redlight_violations_copy = redlight_violations_copy[np.isfinite(redlight_violations_copy['LATITUDE'])]
redlight_violations_copy

speed_violations_copy = speed_violations_copy[np.isfinite(speed_violations_copy['LATITUDE'])]
speed_violations_copy

,ADDRESS,CAMERA ID,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,COORDINATES,coords_x,coords_y
0,5454 W IRVING PARK,CHI050,2014-09-27T00:00:00,85,1.139043e+06,1.926097e+06,41.953330,-87.764267,"{'needs_recoding': False, 'longitude': '-87.76...","(41.953329545,-87.764267264)",-9.769874e+06,5.153991e+06
1,3450 W 71ST,CHI015,2014-10-08T00:00:00,17,1.154378e+06,1.857352e+06,41.764391,-87.709728,"{'needs_recoding': False, 'longitude': '-87.70...","(41.764390957,-87.70972842)",-9.763802e+06,5.125751e+06
2,5454 W IRVING PARK,CHI050,2014-08-11T00:00:00,67,1.139043e+06,1.926097e+06,41.953330,-87.764267,"{'needs_recoding': False, 'longitude': '-87.76...","(41.953329545,-87.764267264)",-9.769874e+06,5.153991e+06
3,5428 S PULASKI,CHI060,2014-09-23T00:00:00,19,1.150589e+06,1.868171e+06,41.794155,-87.723333,"{'needs_recoding': False, 'longitude': '-87.72...","(41.794154584,-87.723333213)",-9.765317e+06,5.130194e+06
4,3810 W BELMONT AVE,CHI089,2014-09-17T00:00:00,1,1.150364e+06,1.921047e+06,41.939258,-87.722781,"{'needs_recoding': False, 'longitude': '-87.72...","(41.939258231,-87.722781428)",-9.765255e+06,5.151885e+06
5,5454 W IRVING PARK,CHI050,2014-09-02T00:00:00,83,1.139043e+06,1.926097e+06,41.953330,-87.764267,"{'needs_recoding': False, 'longitude': '-87.76...","(41.953329545,-87.764267264)",-9.769874e+06,5.153991e+06
6,4124 W FOSTER AVE,CHI003,2014-09-07T00:00:00,124,1.147853e+06,1.934275e+06,41.975605,-87.731670,"{'needs_recoding': False, 'longitude': '-87.73...","(41.975605302,-87.731669795)",-9.766245e+06,5.157326e+06
7,3034 W FOSTER,CHI025,2014-07-16T00:00:00,21,1.155158e+06,1.934429e+06,41.975884,-87.704801,"{'needs_recoding': False, 'longitude': '-87.70...","(41.97588404,-87.704801123)",-9.763254e+06,5.157368e+06
8,5428 S PULASKI,CHI060,2014-09-16T00:00:00,25,1.150589e+06,1.868171e+06,41.794155,-87.723333,"{'needs_recoding': False, 'longitude': '-87.72...","(41.794154584,-87.723333213)",-9.765317e+06,5.130194e+06
9,6226 W IRVING PARK RD,CHI083,2014-10-08T00:00:00,67,1.133995e+06,1.925971e+06,41.953075,-87.782826,"{'needs_recoding': False, 'longitude': '-87.78...","(41.953074826,-87.782825538)",-9.771939e+06,5.153953e+06


In [208]:
redlight_violations_copy["COORDINATES"] = '('+ redlight_violations_copy["LATITUDE"].astype(str) +','+ redlight_violations_copy["LONGITUDE"].astype(str) + ')'
speed_violations_copy["COORDINATES"] = '('+ speed_violations_copy["LATITUDE"].astype(str) +','+ speed_violations_copy["LONGITUDE"].astype(str) + ')'

def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

speed_violations_copy['coords_x'] = speed_violations_copy['COORDINATES'].apply(lambda x: merc(x)[0])
speed_violations_copy['coords_y'] = speed_violations_copy['COORDINATES'].apply(lambda x: merc(x)[1])

redlight_violations_copy['coords_x'] = redlight_violations_copy['COORDINATES'].apply(lambda x: merc(x)[0])
redlight_violations_copy['coords_y'] = redlight_violations_copy['COORDINATES'].apply(lambda x: merc(x)[1])

from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
p = figure(x_range=(-9780000, -9745000), y_range=(5130000, 5160000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

p.circle(x = redlight_violations_copy['coords_x'], y = redlight_violations_copy['coords_y'], fill_color="#FF0000")
p.circle(x = speed_violations_copy['coords_x'], y = speed_violations_copy['coords_y'])

output_notebook()
show(p)

Loading BokehJS ...